In [1]:
# Dependencies
import warnings
import pandas as pd
from pathlib import Path
import panel as pn
import panel.widgets 
warnings.filterwarnings('ignore')
pn.extension()

In [2]:
# Set file paths
take_home_sales_path = Path('Data/take_home_sales.csv', header = 0)
take_home_store_status_path = Path('Data/take_home_store_status.csv', header = 0)
take_home_stores_path = Path('Data/take_home_stores.csv', header = 0)

In [3]:
# Read CSV to DataFrame
take_home_sales_df = pd.read_csv(take_home_sales_path)
take_home_store_status_df = pd.read_csv(take_home_store_status_path)
take_home_stores_df = pd.read_csv(take_home_stores_path)

In [4]:
# Test 'take_home_sales_df' Dataframe
take_home_sales_df.head()

,timestamp|store_id|product_id|units_sold|revenue|cost
0,2018-03-21|25|7WU92401M|2.00 |20.15 |4.86
1,20180320|25|7WU92401XL|2.00 |20.15 |4.44
2,20180320|25|7WU92401XL|2.00 |20.15 |4.44
3,20180402|25|3RD53382|2.00 |24.99 |9.29
4,2018-04-11|25|3WT01010124-5|2.00 |78.80 |9.22


In [5]:
# Test 'take_home_store_status_df' Dataframe
take_home_store_status_df.head()

,tag|description
0,LF|mall
1,RF|outlet
2,JF|stand-alone
3,ZF|closed


In [6]:
# Test 'take_home_stores_df' Dataframe
take_home_stores_df.head()

,store_id|tags
0,25|RF
1,308|LF
2,195|JF
3,242|LF
4,460|RF


# Clean Dataframes

In [7]:
# Break apart column by'|' and rename columns for take_home_sales_cleaned_df
take_home_sales_cleaned_df = pd.DataFrame()
take_home_sales_cleaned_df = take_home_sales_df[take_home_sales_df.columns.values.tolist()[0]].str.split('|',expand=True)
take_home_sales_cleaned_df.columns = take_home_sales_df.columns.values.tolist()[0].split('|')

In [8]:
take_home_sales_cleaned_df.head(10)

,timestamp,store_id,product_id,units_sold,revenue,cost
0,2018-03-21,25,7WU92401M,2.00,20.15,4.86
1,20180320,25,7WU92401XL,2.00,20.15,4.44
2,20180320,25,7WU92401XL,2.00,20.15,4.44
3,20180402,25,3RD53382,2.00,24.99,9.29
4,2018-04-11,25,3WT01010124-5,2.00,78.80,9.22
5,2018-04-11,25,3WT01010126-7,2.00,78.80,9.21
6,20180326,25,2WV26301L,2.00,47.98,22.99
7,20180326,25,2WV26301L,2.00,22.99,12.22
8,20180326,25,2WV26301L,2.00,22.99,12.22
9,20180326,25,2WV26301M,2.00,22.99,12.21


In [9]:
# Break apart column by'|' and rename columns for take_home_store_status_df
take_home_store_status_cleaned_df = take_home_store_status_df[take_home_store_status_df.columns.values.tolist()[0]].str.split('|',expand=True)
take_home_store_status_cleaned_df.columns = take_home_store_status_df.columns.values.tolist()[0].split('|')
take_home_store_status_cleaned_df = take_home_store_status_cleaned_df.set_index('tag')
take_home_store_status_cleaned_df.head()

,description
tag,
LF,mall
RF,outlet
JF,stand-alone
ZF,closed


In [10]:
# Break apart column by'|' and rename columns for take_home_stores_df
take_home_stores_cleaned_df = take_home_stores_df[take_home_stores_df.columns.values.tolist()[0]].str.split('|',expand=True)
take_home_stores_cleaned_df.columns = take_home_stores_df.columns.values.tolist()[0].split('|')
take_home_stores_cleaned_df.head(10)

,store_id,tags
0,25,RF
1,308,LF
2,195,JF
3,242,LF
4,460,RF
5,325,RF
6,405,RF
7,1908,JF
8,358,ZF
9,292,LF


In [11]:
# Convert take_home_store_status_df to dictionary for mapping of description column onto take_home_stores_df
take_home_store_status_cleaned_dict = take_home_store_status_cleaned_df.to_dict()
#take_home_store_status_cleaned_dict['description']
take_home_stores_cleaned_df['description'] = take_home_stores_cleaned_df['tags'].map(take_home_store_status_cleaned_dict['description'])
take_home_stores_cleaned_df.head(10)

,store_id,tags,description
0,25,RF,outlet
1,308,LF,mall
2,195,JF,stand-alone
3,242,LF,mall
4,460,RF,outlet
5,325,RF,outlet
6,405,RF,outlet
7,1908,JF,stand-alone
8,358,ZF,closed
9,292,LF,mall


In [12]:
# Create a new dataframe with ids for only 'opened stores' by filtering out rows with 'closed' stores
open_store_ids_df = take_home_stores_cleaned_df[take_home_stores_cleaned_df['description']!='closed']
open_store_ids_df.reset_index()
open_store_ids_df.head(10)

,store_id,tags,description
0,25,RF,outlet
1,308,LF,mall
2,195,JF,stand-alone
3,242,LF,mall
4,460,RF,outlet
5,325,RF,outlet
6,405,RF,outlet
7,1908,JF,stand-alone
9,292,LF,mall
10,100,JF,stand-alone


In [13]:
# Use open_store_ids_df to filtered out closed stores of take_home_store_status_cleaned_df
open_take_home_sales_cleaned_df = pd.DataFrame()
open_take_home_sales_cleaned_df = take_home_sales_cleaned_df[take_home_sales_cleaned_df['store_id'].isin(open_store_ids_df['store_id'])]
open_take_home_sales_cleaned_df.reset_index()
open_take_home_sales_cleaned_df.head(10)

,timestamp,store_id,product_id,units_sold,revenue,cost
0,2018-03-21,25,7WU92401M,2.00,20.15,4.86
1,20180320,25,7WU92401XL,2.00,20.15,4.44
2,20180320,25,7WU92401XL,2.00,20.15,4.44
3,20180402,25,3RD53382,2.00,24.99,9.29
4,2018-04-11,25,3WT01010124-5,2.00,78.80,9.22
5,2018-04-11,25,3WT01010126-7,2.00,78.80,9.21
6,20180326,25,2WV26301L,2.00,47.98,22.99
7,20180326,25,2WV26301L,2.00,22.99,12.22
8,20180326,25,2WV26301L,2.00,22.99,12.22
9,20180326,25,2WV26301M,2.00,22.99,12.21


In [14]:
# Remove invalid product ids in open_take_home_sales_cleaned_df
valid_df = open_take_home_sales_cleaned_df[~open_take_home_sales_cleaned_df['product_id'].str.contains('-' , '/')]
valid_df.reset_index()
valid_df.head(10)

,timestamp,store_id,product_id,units_sold,revenue,cost
0,2018-03-21,25,7WU92401M,2.00,20.15,4.86
1,20180320,25,7WU92401XL,2.00,20.15,4.44
2,20180320,25,7WU92401XL,2.00,20.15,4.44
3,20180402,25,3RD53382,2.00,24.99,9.29
6,20180326,25,2WV26301L,2.00,47.98,22.99
7,20180326,25,2WV26301L,2.00,22.99,12.22
8,20180326,25,2WV26301L,2.00,22.99,12.22
9,20180326,25,2WV26301M,2.00,22.99,12.21
10,20180326,25,2WV26301S,2.00,20.29,12.26
11,2018-03-23,25,3WB22201L,2.00,22.99,12.23


In [15]:
# Date level cleaning to same time format
valid_time_df = pd.DataFrame()
valid_time_df = valid_df
valid_time_df['timestamp'] = pd.to_datetime(valid_time_df['timestamp'])
valid_time_df['timestamp'] = valid_time_df['timestamp'].map(lambda x: x.strftime("%m-%d-%Y"))
valid_time_df.head(10)

,timestamp,store_id,product_id,units_sold,revenue,cost
0,03-21-2018,25,7WU92401M,2.00,20.15,4.86
1,03-20-2018,25,7WU92401XL,2.00,20.15,4.44
2,03-20-2018,25,7WU92401XL,2.00,20.15,4.44
3,04-02-2018,25,3RD53382,2.00,24.99,9.29
6,03-26-2018,25,2WV26301L,2.00,47.98,22.99
7,03-26-2018,25,2WV26301L,2.00,22.99,12.22
8,03-26-2018,25,2WV26301L,2.00,22.99,12.22
9,03-26-2018,25,2WV26301M,2.00,22.99,12.21
10,03-26-2018,25,2WV26301S,2.00,20.29,12.26
11,03-23-2018,25,3WB22201L,2.00,22.99,12.23


In [16]:
# Convert revenue to numeric type across the entire file
valid_time_df['revenue'] = valid_time_df['revenue'].str.replace("-","")
valid_time_df['revenue'] = pd.to_numeric(valid_time_df['revenue'])
valid_time_df.head()

,timestamp,store_id,product_id,units_sold,revenue,cost
0,03-21-2018,25,7WU92401M,2.00,20.15,4.86
1,03-20-2018,25,7WU92401XL,2.00,20.15,4.44
2,03-20-2018,25,7WU92401XL,2.00,20.15,4.44
3,04-02-2018,25,3RD53382,2.00,24.99,9.29
6,03-26-2018,25,2WV26301L,2.00,47.98,22.99


In [17]:
# Convert revenue to numeric type across the entire file
valid_time_df['store_id'] = pd.to_numeric(valid_time_df['store_id'])
valid_time_df.head()

,timestamp,store_id,product_id,units_sold,revenue,cost
0,03-21-2018,25,7WU92401M,2.00,20.15,4.86
1,03-20-2018,25,7WU92401XL,2.00,20.15,4.44
2,03-20-2018,25,7WU92401XL,2.00,20.15,4.44
3,04-02-2018,25,3RD53382,2.00,24.99,9.29
6,03-26-2018,25,2WV26301L,2.00,47.98,22.99


# Deliverables

In [18]:
# Total - a sum of revenue across the entire file
Total = round(valid_time_df['revenue'].sum())
print("Total Revenue is " + str(Total) + " Dollars")

Total Revenue is 266601549 Dollars


In [19]:
# Store level - a sum of revenue grouped by stores
Store_Level_Total = valid_time_df.groupby(by = ['store_id']).sum().reset_index().sort_values('store_id')
Store_Level_Total.head(10)

,store_id,revenue
0,5,28292.20
1,10,33165.82
2,12,22081.70
3,15,30633.75
4,22,30906.35
5,25,47509.67
6,35,8880.36
7,40,11724.01
8,42,35480.31
9,45,35700.02


In [25]:
date_level_df = pd.DataFrame()
date_level_df = open_take_home_sales_cleaned_df
date_level_df['timestamp'] = pd.to_datetime(date_level_df['timestamp'])
date_level_df['revenue'] = date_level_df['revenue'].str.replace("-","")
date_level_df['revenue'] = pd.to_numeric(date_level_df['revenue'])

datetime_series = date_level_df['timestamp']
datetime_index = pd.DatetimeIndex(datetime_series.values)
date_level_df_2 = date_level_df.set_index(datetime_index)
date_level_df_2.drop('timestamp',axis=1,inplace=True)

date_level_df_2.head(10)


,store_id,product_id,units_sold,revenue,cost
2018-03-21,25,7WU92401M,2.00,20.15,4.86
2018-03-20,25,7WU92401XL,2.00,20.15,4.44
2018-03-20,25,7WU92401XL,2.00,20.15,4.44
2018-04-02,25,3RD53382,2.00,24.99,9.29
2018-04-11,25,3WT01010124-5,2.00,78.80,9.22
2018-04-11,25,3WT01010126-7,2.00,78.80,9.21
2018-03-26,25,2WV26301L,2.00,47.98,22.99
2018-03-26,25,2WV26301L,2.00,22.99,12.22
2018-03-26,25,2WV26301L,2.00,22.99,12.22
2018-03-26,25,2WV26301M,2.00,22.99,12.21


In [39]:
# Sum all days together from all stores
Date_Level_Total_df = date_level_df_2.groupby(level=0)['revenue'].sum().reset_index().sort_values('index')
Date_Level_Total_df.columns = ['timestamp','revenue']
#Date_Level_Total_df.head(10)


In [38]:
# Groupby the first Thursday of Every Week
Date_Level_Total_df_2 = Date_Level_Total_df.groupby([pd.Grouper(key='timestamp', freq='W-THU')])['revenue'].sum().reset_index().sort_values('timestamp')
Date_Level_Total_df_2.head(100)

,timestamp,revenue
0,2018-03-22,4.718528e+07
1,2018-03-29,7.471989e+07
2,2018-04-05,7.760308e+07
3,2018-04-12,7.494551e+07
4,2018-04-19,1.713724e+07
